* Program contains 2 or more part that run concurrently. Each part is called thread and each thread defines separate path of execution.
* Process based vs thread based multitasking
    - Process is program in execution. Allows 2 or more program to run concurrently. Running Java compiler at the same time with text editor. Program is smallest unit of code that can be dispatched by the scheduler.
    - Thread based multitasking. Thread is smallest unit of dispatchable code. Meaning single program can perform 2 or more task simultaneously. Text editor can do spell check, printing at the same time as long as both action are in separate thread.
    - Process are heavy weight task that needs own separate address space. Inter process communication is expensive and limited. Context switch of process is also expensive.
    - Thread are lighter weight, they share same address space. interthread communication and context switch of thread is NOT expensive.
    - Process based multitasking is not under Java's control but thread based is.
* Single threaded system has event loop with polling. Single thread of control runs in infinite loop, polling single event queue to decide what to do next. When queue says something to be done, event loop dispatches control to appropriate event handler. Until this event handler returns nothing can be done. Sometimes that event handler waiting for resource and all other task has to wait.
* In multithreaded model there is no loop/polling. One thread can pause without stopping other part of program. All other thread can continue execution.
* In single core system, concurrently executing thread shares CPU time. In multi-core system it is possible that 2 core can execute simultaneously.
* Thread has several states: running, ready to run, suspended (temporary holds its activity), resumed (pick up from where it left off), blocked (waiting for resource).

### Thread priority
* Java assign priority to each thread. It is integer that designate relative priority from one thread to other. It is useful during context switch (when we decide to switch from one running thread to other)
    - Thread can voluntarily relinquish control: DOne by explicitly yielding, sleeping or blocking on pending I/O. For such times all other thread are examined and highest priority thread is ready to get CPU.
    - Thread can be preempted by higher priority thread.  (preemptive multitasking)
* Thread with same priority may share CPU slices (windows) or voluntarily yield the control to peers

### Synchronization
* Multithread introduce asynchronous behavior to program, we need way to enforce synchronization.
* When 2 thread share some data structure we need some mechanism for synchronization. Monitor is used by Java. Monitor can be think as small box where only 1 thread can fit. Once 1 thread enter monitor all other thread has to wait until it exit. It protect shared asset from being manipulated by more than one thread at a time.
* Every object has its own implicit monitor that is automatically entered when one of the object's synchronized method is called. Once thread is inside synchronize method, no other thread can call any other synchronized method on same object.

### Thread class
* Java's multithreaded system is built on `Thread` class, its method and its companion interface `Runnable`.
* To create thread, your program will either extend Thread or implement the runnable interface.
* Thread class defines several methods,
![thread](images/thread.jpg)

### The main thread
* When java program startup one thread begins running immediately. This is called main thread.
    - It is the thread from which other child thread will be spawned.
    - It must be the last thread to finish execution because it performs various shutdown actions.
```
static Thread currentThread()

static void sleep(long milliseconds) throws InterruptedException
static void sleep(long milliseconds, int nanoseconds) throws InterruptedException

final void setName(String threadName)
final String getName()
```
* Obtain reference of currently running thread. We can call this method to get reference to main thread.

```
public class ThreadDemo {
	public static void main(String[] args) {
		Thread t = Thread.currentThread();
		
		System.out.println("Current thread " + t);
		
		t.setName("My Thread"); // set the name of thread.
		
		System.out.println("After name change " + t);

		try {
			for (int n = 5; n > 0; n--) {
				System.out.println(n);
				Thread.sleep(1000); // Causes the thread from which it is called to suspend execution 
                                    // for specified period of milliseconds
			}
			// Sleep() method in Thread might throw an InterruptedException.
			// Happens when some other thread want to interrupt sleeping one
		} catch (InterruptedException e){
				System.out.println("Main thread interrupted");
		}
	}
}
```
* output
```
Current thread Thread[main,5,main] 
After name change Thread[My Thread,5,main]
5
4
3
2
1
```
* [My Thread,5,main] is name of the thread, priority and group name. Thread group is data structure that controls the state of a collection of threads as whole.

### Creating Thread
* Instantiate object of type thread.
    - implement Runnable interface
    - Extend the Thread class
    
#### Implement Runnable interface
* Create class that implements Runnable interface. To implement it class only has to implement method `run()`.

```
public void run()
```
* Inside run we define code that constitutes new thread.
* After creating class we instantiate an object of type Thread from within that class.

```
Thread(Runnable threadOb, String threadName)
```

* threadob is instance of a class that implements Runnable. This defines where execution of thread will begin.
* We have call `start()` to run the thread. `start()` will internally call `run()`.

```
public class NewThread implements Runnable{
	Thread t;
	
	NewThread() {
		// create new thread
		t = new Thread(this, "Demo Thread"); // new thread will execute run method on this object
		System.out.println("Chile thread" + t);
		t.start(); // start the thread
	}
	
	public void run() {
		try {
			for (int i = 5; i > 0; i --) {
				System.out.println("Child thread" + i);
				Thread.sleep(1000);
			}
		} catch(InterruptedException e) {
			System.out.println("Child interrupted");
		}
		System.out.println("Exit child thread");
	}
}


public class Demo {
	public static void main(String[] args) {
		new NewThread();
		
		try {
			for (int i = 0; i < 5; i++) {
				System.out.println(i);
				Thread.sleep(1000);
			}
		} catch (InterruptedException e) {
			System.out.println("Main thread interupted");
		}
		System.out.println("Main exited");
	}
}
```

* Output

```
Chile threadThread[Demo Thread,5,main]
0
Child thread5
1
Child thread4
2
Child thread3
3
Child thread2
4
Child thread1
Main exited
Exit child thread
```

#### Extending Thread
* Create new class that extends thread. Then create the instance of class. Extending class must override `run()` method, which is entry point to new thread. It must call start() to begin execution of the new thread.

```
public class NewThread extends Thread {
	NewThread() {
		super("Demo Thread"); // invokes constructor public Thread(String threadName)
		System.out.println("Chile thread" + this);
		start(); // start the thread
	}
	
	public void run() {
		try {
			for(int i = 5; i > 0; i--) {
				System.out.println("Child thread" + i);
				Thread.sleep(500);
			}
		} catch (InterruptedException e) {
			System.out.println("Child interrupted");
		}
		System.out.println("Child done");
	}
}


public class Demo {
	public static void main(String[] args) {
		new NewThread();
		
		try {
			for (int i = 0; i < 5; i++) {
				System.out.println(i);
				Thread.sleep(1000);
			}
		} catch (InterruptedException e) {
			System.out.println("Main thread interupted");
		}
		System.out.println("Main exited");
	}
}

```
* Output

```
Chile threadThread[Demo Thread,5,main]
0
Child thread5
Child thread4
1
Child thread3
Child thread2
2
Child thread1
Child done
3
4
Main exited
```

### `isAlive()` `join()`
* `isAlive()` returns true if the thread upon which it is called still running or false.
* `join()` waits until the thread on which it is called terminates. Calling thread wait until called thread joins back. Also we can specify max time for which we want to wait for other thread to terminate.

```
public class Demo {
	public static void main(String[] args) {
		NewThread ob1 = new NewThread("one");
		NewThread ob2 = new NewThread("two");
		NewThread ob3 = new NewThread("three");
		
		System.out.println(ob1.t.isAlive());
		System.out.println(ob2.t.isAlive());
		System.out.println(ob3.t.isAlive());
		try {
			ob1.t.join();
			ob2.t.join();
			ob3.t.join();
		} catch (InterruptedException e) {
			System.out.println("Main thread interupted");
		}
		System.out.println("Main exited");
	}
}

```

### Thread priority
* Used by thread scheduler to decide which thread should be allowed to run.
* To set priority of thread use,

```
final void setPriority(int level)
```
* level must be between 1 and 10.
* To obtain current priority we can call `getPriority()` method on thread.


### Synchronization
* When 2 or more thread wants to use shared resources, they need to make sure that th resource will be used by one thread at a time.
* monitor is an object that is used as mutually exclusive lock. 1 thread can own monitor at given time. When thread acquire lock it said to be in monitor. ALl other thread must has to wait until entered thread exits.
* To enter object's monitor call the method which has been modified with synchronized keyword. 

```
class CallMe {
	void call(String msg) {
		System.out.println("[" + msg);
		try {
			Thread.sleep(1000);
		} catch (InterruptedException e) {
			System.out.println("Interrupted");
		}
		System.out.println("]");
	}
}

class Caller implements Runnable {
	String msg;
	CallMe target;
	Thread t;
	
	Caller(CallMe targ, String s) {
		target = targ;
		msg = s;
		t = new Thread(this);
		t.start();
	}
	public void run() {
		target.call(msg);
	}
}


public class Sync {
	public static void main(String[] args) {
		CallMe target = new CallMe();
		
		Caller ob1 = new Caller(target, "Hello");
		Caller ob2 = new Caller(target, "Synchronized");
		Caller ob3 = new Caller(target, "World");
		
		try {
			ob1.t.join();
			ob2.t.join();
			ob3.t.join();
		} catch(InterruptedException e) {
			System.out.println("Interrupted");
		}
	}

}

```
* Output

```
[Hello
[World
[Synchronized
]
]
]
```
* This is race condition, 3 thread is racing each other to complete method
* We have to provide serialize access to call(). Meaning 1 thread can access it at a time.
* Just add keyword `synchronized` in front of method.

```
synchronized void call(String msg) {
```
* This prevents from, entering multiple thread in this method.
* To synchronize access to object of class, put the calls to the method defined by this class inside `synchronized` block.

```
synchronized(objRef) {
    //statement to be synchronized
}
```
* objRef is reference to the object being synchronized. Synchronized block ensures that a call to a synchronized method that is member of objRef class occurs only after te current thread has successfully entered objRef's monitor.

```
class Caller implements Runnable {
	String msg;
	CallMe target;
	Thread t;
	
	Caller(CallMe targ, String s) {
		target = targ;
		msg = s;
		t = new Thread(this);
		t.start();
	}
	public void run() {
		synchronized(target) {
			target.call(msg);
		}
	}
}
```

### Interthread communication
* Polling is created by loop that used to check some condition and once the condition is true, appropriate action is taken. This waste CPU time.
* To avoid it java has inter-process communication via `wait()`, `notify()`, `notifyAll()` methods. They can call within synchronized context. 
* `wait()` tells calling thread to give up the monitor and goto sleep until some other thread enters the same monitor and calls `notify()` or `notifyAll()`.
* `notify()` wakes up a thread that called `wait()` on the same object,
* `notfyAll()`wakes up all threads that called wait() on the same object. One of the thread will be granted access.

```
final void wait() throws InterruptedException
final void notify()
final void notifyAllee()
```


```
class Q {
	int n;
	boolean valueSet = false;
	synchronized int get() {
		while(!valueSet) { 
			try {
				wait();
			} catch (InterruptedException e) {
				System.out.println("Intrrupted");
			}
		}
		System.out.println("Got " + n);
		valueSet = false;
		notify();
		return n;
	}
	
	synchronized void put(int n) {
		while(valueSet) {
			try {
				wait();
			} catch (InterruptedException e) {
				System.out.println("Interrupted");
			}
		}
		
		this.n= n;
		valueSet = true;
		System.out.println("Put " + n);
		notify();
	}
}

class Producer implements Runnable {
	Q q;
	
	Producer(Q q) {
		this.q = q;
		new Thread(this, "Producer").start();
	}
	
	public void run() {
		int i = 0;
		while(true) {
			q.put(i++);
		}
	}
}

class Consumer implements Runnable {
	Q q;
	
	Consumer(Q q) {
		this.q = q;
		new Thread(this, "Consumer").start();
	}
	public void run() {
		while(true) {
			q.get();
		}
	}
	
}
public class PC {

	public static void main(String[] args) {
		Q q = new Q();
		new Producer(q);
		new Consumer(q);

	}

}
```

### Deadlock
* When 2 thread has circular dependency on pair of synchronized object.
* One thread enter monitor on object X, another enter monitor of object Y. If the thread in X tries to call  any synchronized method on Y it will be blocked. When thread in Y, tries to call synchronized object in X. It leads to Deadlock.
* 